In [1]:
import numpy as np
import skimage as ski
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

In [2]:
from optimizers import ista_L1
from operators import Radon, L1_norm
from utils import get_phantom

dim = 100
pre_compute = {}
phantom = get_phantom(dim)

def plot_lamda(lamda, noise_lvl, num_theta):
    theta = np.linspace(0,180, endpoint= False, num=num_theta)
    R = Radon(theta=theta)
    if (lamda, noise_lvl, num_theta) in pre_compute:
        u, energy_hist = pre_compute[(lamda, noise_lvl, num_theta)]
    else:
        sinogram = R(phantom) + np.random.normal(loc = 0, scale= noise_lvl, size = [dim,num_theta])
        def energy_fun(u):
            return np.linalg.norm(R(u) - sinogram)**2 + lamda * L1_norm()(u)
        
        ista = ista_L1(R, R.inv(sinogram), sinogram, t=0.1, lamda=lamda, energy_fun=energy_fun, verbosity = 0.)
        ista.solve()
        
        
        u = ista.u
        energy_hist = ista.energy_hist
        pre_compute[(lamda, noise_lvl, num_theta)] = [u, energy_hist]
    
    fig, ax = plt.subplots(1,2, figsize=(20,5))
    ax[0].imshow(u)
    ax[1].plot(energy_hist, linewidth=3, color='xkcd:sky')
    ax[1].set_title('Energy')
    ax[1].set_xlabel('Iteration')


l_slider = widgets.FloatSlider(min = 0, max = 1., step = 0.1, value = 0, continuous_update = False)
s_slider = widgets.FloatSlider(min = 0, max = .1, step = 0.01, value = 0, continuous_update = False)
t_slider = widgets.IntSlider(min = 1, max = 100, step = 0.01, value = 100, continuous_update = False)
interactive_plot = interactive(plot_lamda, lamda = l_slider, noise_lvl = s_slider, num_theta = t_slider)
display(interactive_plot)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='lamda', max=1.0), FloatSlid…

In [3]:
from optimizers import ista_L1, split_Bregman_TV
from operators import Radon, L1_norm, TV
from utils import get_phantom

dim = 100
num_theta = 100
pre_compute = {}
phantom = get_phantom(dim)

def plot_lamda(lamda, noise_lvl, max_angle):
    inp = (lamda, noise_lvl, max_angle)
    theta = np.linspace(0,max_angle, endpoint= False, num=num_theta)
    R = Radon(theta=theta)
    if inp in pre_compute:
        u, energy_hist = pre_compute[inp]
    else:
        sinogram = R(phantom) + np.random.normal(loc = 0, scale= noise_lvl, size = [dim,num_theta])
        def energy_fun(u):
            return np.linalg.norm(R(u) - sinogram)**2 + lamda * TV()(u)
        
        sBTV = split_Bregman_TV(R, sinogram, R.inv(sinogram), gamma=1.0, 
                        energy_fun=energy_fun, lamda = lamda,
                        max_it = 10,
                        max_inner_it = 2, verbosity = 0)
        sBTV.solve()
        
        u = sBTV.u
        energy_hist = sBTV.energy_hist
        pre_compute[inp] = [u, energy_hist]
    
    fig, ax = plt.subplots(1,2, figsize=(20,5))
    ax[0].imshow(u)
    ax[1].plot(energy_hist, linewidth=3, color='xkcd:sky')
    ax[1].set_title('Energy')
    ax[1].set_xlabel('Iteration')


l_slider = widgets.FloatSlider(min = 0.0, max = .1, step = 0.01, value = 0, continuous_update = False)
s_slider = widgets.FloatSlider(min = 0, max = .05, step = 0.01, value = 0, continuous_update = False)
a_slider = widgets.IntSlider(min = 1, max = 180, step = 10, value = 180, continuous_update = False)
interactive_plot = interactive(plot_lamda, 
                               lamda = l_slider, 
                               noise_lvl = s_slider,
                               max_angle = a_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='lamda', max=0.1, step=0.01)…

In [4]:
approx = []
for lamda in range(20):
    approx.append(ista(y, radon, radon_adjoint, soft_shrinkage, t=0.0003, lamda = lamda, iter = 50))



KeyboardInterrupt



In [ ]:
def plot_lamda(idx):
    
    fig = plt.figure()
    plt.imshow(approx[idx])


slider = widgets.IntSlider(min = 0, max = 19, step = 1, value = 0, continuous_update = True)
interactive_plot = interactive(plot_lamda, idx = slider)
display(interactive_plot)

In [ ]:
plt.imshow(ski.transform.iradon(y,theta))
plt.figure()
plt.imshow(approx)

In [ ]:
alpha = 3
x = np.linspace(-10,10,num=100)
prox_x = soft_shrinkage(x, alpha)
plt.plot(x, prox_x)